In [5]:
import csv
from datetime import time
import keras
import tf
from keras.callbacks import TensorBoard

from scipy.io import loadmat


import os
import shutil
%matplotlib inline

train_dir = '../data/processed/train'

annots = loadmat('../data/raw/imagelabels.mat')

type(annots['labels']),annots['labels'].shape
type(annots['labels'][0][0]),annots['labels'][0][0].shape


def sort_to_classes_folder():
    raw_img_folder = "../data/raw/jpg"
    for image_name in os.listdir(raw_img_folder):
        img_idx = int(image_name.strip("image_").strip(".jpg"))
        
        label = annots['labels'][0][img_idx-1]
    
        class_dir_name = "../data/processed/no_split" + str(label)
        if not os.path.exists(class_dir_name):
            os.mkdir(class_dir_name)
            
        src_image = raw_img_folder + "/" + image_name
        dest = class_dir_name + "/" + image_name
        
        shutil.copyfile(src_image, dest)
 

Create a csv with class_id, label.
Labels taken from here: https://gist.github.com/jimgoo/0179e52305ca768a601f#gistcomment-1486680


In [6]:
label_names = ['pink primrose', 'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea', 'english marigold', 'tiger lily', 'moon orchid', 'bird of paradise', 'monkshood', 'globe thistle', 'snapdragon', "colt's foot", 'king protea', 'spear thistle', 'yellow iris', 'globe-flower', 'purple coneflower', 'peruvian lily', 'balloon flower', 'giant white arum lily', 'fire lily', 'pincushion flower', 'fritillary', 'red ginger', 'grape hyacinth', 'corn poppy', 'prince of wales feathers', 'stemless gentian', 'artichoke', 'sweet william', 'carnation', 'garden phlox', 'love in the mist', 'mexican aster', 'alpine sea holly', 'ruby-lipped cattleya', 'cape flower', 'great masterwort', 'siam tulip', 'lenten rose', 'barbeton daisy', 'daffodil', 'sword lily', 'poinsettia', 'bolero deep blue', 'wallflower', 'marigold', 'buttercup', 'oxeye daisy', 'common dandelion', 'petunia', 'wild pansy', 'primula', 'sunflower', 'pelargonium', 'bishop of llandaff', 'gaura', 'geranium', 'orange dahlia', 'pink-yellow dahlia?', 'cautleya spicata', 'japanese anemone', 'black-eyed susan', 'silverbush', 'californian poppy', 'osteospermum', 'spring crocus', 'bearded iris', 'windflower', 'tree poppy', 'gazania', 'azalea', 'water lily', 'rose', 'thorn apple', 'morning glory', 'passion flower', 'lotus', 'toad lily', 'anthurium', 'frangipani', 'clematis', 'hibiscus', 'columbine', 'desert-rose', 'tree mallow', 'magnolia', 'cyclamen ', 'watercress', 'canna lily', 'hippeastrum ', 'bee balm', 'ball moss', 'foxglove', 'bougainvillea', 'camellia', 'mallow', 'mexican petunia', 'bromelia', 'blanket flower', 'trumpet creeper', 'blackberry lily']

processed_path = "../data/processed/"
with open(processed_path + 'id_label.csv','w') as file:
    wr = csv.writer(file)
    for idx, label in enumerate(label_names):
        line =[idx+1,label]
        print(line)
        wr.writerow(line)


[1, 'pink primrose']
[2, 'hard-leaved pocket orchid']
[3, 'canterbury bells']
[4, 'sweet pea']
[5, 'english marigold']
[6, 'tiger lily']
[7, 'moon orchid']
[8, 'bird of paradise']
[9, 'monkshood']
[10, 'globe thistle']
[11, 'snapdragon']
[12, "colt's foot"]
[13, 'king protea']
[14, 'spear thistle']
[15, 'yellow iris']
[16, 'globe-flower']
[17, 'purple coneflower']
[18, 'peruvian lily']
[19, 'balloon flower']
[20, 'giant white arum lily']
[21, 'fire lily']
[22, 'pincushion flower']
[23, 'fritillary']
[24, 'red ginger']
[25, 'grape hyacinth']
[26, 'corn poppy']
[27, 'prince of wales feathers']
[28, 'stemless gentian']
[29, 'artichoke']
[30, 'sweet william']
[31, 'carnation']
[32, 'garden phlox']
[33, 'love in the mist']
[34, 'mexican aster']
[35, 'alpine sea holly']
[36, 'ruby-lipped cattleya']
[37, 'cape flower']
[38, 'great masterwort']
[39, 'siam tulip']
[40, 'lenten rose']
[41, 'barbeton daisy']
[42, 'daffodil']
[43, 'sword lily']
[44, 'poinsettia']
[45, 'bolero deep blue']
[46, 'wal

In [7]:

def rename_image_plain_id():
    
    raw_img_folder = "../data/raw/jpg"
    
    for image_name in os.listdir(raw_img_folder):
        
        img_idx = int(image_name.strip("image_").strip(".jpg"))
        id_folder = "../data/processed/images_by_id"
        
        if not os.path.exists(id_folder):
            os.mkdir(id_folder)
            
        src_image = raw_img_folder + "/" + image_name
        dest = id_folder + "/" + str(img_idx) + ".jpg"
        
        shutil.copyfile(src_image, dest)
        
rename_image_plain_id()

Evaluate the given split files and distribute the data accordingly in the processed directory.

In [1]:
import os
import shutil
from scipy.io import loadmat

from src.data.AugmentedManifest import AugmentedManifest

s3_bucket_base_url = "s3://sagemaker-data-prod/oxford-flower-102/"

folder_dict = {'trnid':'train', 'valid':'valid' , 'tstid':'test'}

annotations = loadmat('../data/raw/imagelabels.mat')
splits = loadmat('../data/raw/setid.mat')
splits.keys()
type(splits['trnid']),splits['trnid'].shape
type(splits['tstid']),splits['tstid'].shape
type(splits['valid']),splits['valid'].shape


# potentially overwrite
for split_id, folder_name in folder_dict.items():
    
    target_dir = "../data/processed/" + folder_name
    s3_image_base_url = s3_bucket_base_url
    
    shutil.rmtree(target_dir, ignore_errors=True)
    os.makedirs(target_dir)
    
    manifest = AugmentedManifest(target_dir + "/_manifest.jsonl")
    
    for img_id in splits[split_id][0]:
        src = "../data/processed/images_by_id/{}.jpg".format(img_id)
        class_id = annotations['labels'][0][img_id - 1]           
        class_folder = "../data/processed/{}/{}".format(folder_name, str(class_id))
        
        if not os.path.exists(class_folder):
            os.makedirs(class_folder)
            
        img_name = "{}.jpg".format(img_id)
        full_image_path = os.path.join(class_folder, img_name) 
        shutil.copyfile(src, full_image_path)
        
    
        s3_image_path = full_image_path.replace("../data/processed/", s3_image_base_url)
        manifest.add_entry({"source-ref":s3_image_path, "label":int(class_id)})

    manifest.finished_writing()

Explore Augmentations

In [ ]:
from keras_preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

for img in aug.flow_from_directory("../data/processed/train", save_to_dir="../data/processed/train_aug"):
    pass

Define an actual model - I am so excited !!!


In [1]:
from tensorflow import keras

model = keras.models.Sequential()

# conv
model.add(keras.layers.Conv2D(32, (3,3), activation='elu', input_shape=(150,150,3)))
model.add(keras.layers.MaxPooling2D(2,2))

model.add(keras.layers.Flatten())

# Fully connected
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(102, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimzer = keras.optimizers.RMSProp(lr=1e-4), metrics=['acc'])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-937584d339da>", line 16, in <module>
    model.compile(loss='categorical_crossentropy', optimzer = keras.optimizers.RMSProp(lr=1e-4), metrics=['acc'])
AttributeError: module 'tensorflow_core.python.keras.api._v2.keras.optimizers' has no attribute 'RMSProp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2040, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1101, in ge

AttributeError: module 'tensorflow_core.python.keras.api._v2.keras.optimizers' has no attribute 'RMSProp'

Train the model !


In [1]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from datetime import time
import numpy as np


from matplotlib import pyplot as plt
%matplotlib

model = tf.keras.models.Sequential()

img_height = img_width = 180

# conv
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(img_height, img_width  , 3)))
model.add(tf.keras.layers.MaxPooling2D(2,2))
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2,2))
model.add(tf.keras.layers.Flatten())

# Fully connected
#model.add(tf.keras.layers.Dense(256, activation='sigmoid'))
model.add(tf.keras.layers.Dense(128,activation = 'sigmoid'))
#model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(128, ))
model.add(tf.keras.layers.Dense(102, activation='softmax'))


model.compile(optimizer=tf.keras.optimizers.RMSprop(),  # Optimizer
              # Loss function to minimize
              loss=tf.keras.losses.CategoricalCrossentropy(),
              # List of metrics to monitor
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

test_dir = '../data/processed/test'
train_dir = '../data/processed/train'
valid_dir = '../data/processed/valid'
train_valid_combined_dir = '../data/processed/train-valid-combine'


train_data_generator = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
 	height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, vertical_flip=True, fill_mode="nearest", 
                                           validation_split=0.2)


# train_data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)

class_names = [str(i+1) for i in range(102)]
train_data = train_data_generator.flow_from_directory(train_valid_combined_dir, 
                                                      subset='training', seed=42, target_size=(img_height, img_width), 
                                                      classes=class_names, class_mode='categorical')

validation_data = train_data_generator.flow_from_directory(train_valid_combined_dir, 
                                                           subset='validation',seed=42 ,target_size=(img_height, img_width), 
                                                           class_mode='categorical')

logdir = "reports/figures/tensorboard/{}".format(time())
#tensorboard_callback = TensorBoard(logdir, histogram_freq=0)

history = model.fit(train_data, epochs=7)


model.save("../models/oxford_flowers_v1.hdf5")
model.summary()


test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width) ,
        classes=class_names,
        class_mode='categorical',
        shuffle=False)


pred = model.evaluate(test_generator, steps=len(test_generator), verbose=2)

print(pred)

Using matplotlib backend: MacOSX
Found 1654 images belonging to 102 classes.
Found 386 images belonging to 102 classes.
  ...
    to  
  ['...']
Train for 52 steps
Epoch 1/7
52/52 [==============================] - 81s 2s/step - loss: 4.7380 - categorical_accuracy: 0.0193
Epoch 2/7
52/52 [==============================] - 81s 2s/step - loss: 4.2929 - categorical_accuracy: 0.0290
Epoch 3/7
52/52 [==============================] - 79s 2s/step - loss: 3.9392 - categorical_accuracy: 0.0647
Epoch 4/7
52/52 [==============================] - 79s 2s/step - loss: 3.6982 - categorical_accuracy: 0.1040
Epoch 5/7
52/52 [==============================] - 61s 1s/step - loss: 3.5092 - categorical_accuracy: 0.1391
Epoch 6/7
52/52 [==============================] - 59s 1s/step - loss: 3.3665 - categorical_accuracy: 0.1596
Epoch 7/7
52/52 [==============================] - 61s 1s/step - loss: 3.2251 - categorical_accuracy: 0.1856
Model: "sequential"
_____________________________________________________

In [ ]:
import tensorflow as tf

print(tf.__version__)
